In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [6]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [7]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [19]:
df_green.createOrReplaceTempView('green')

In [20]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    green
WHERE
    pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [21]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2062-08-15 00:00:00|   7|               3.6|             1|
|2020-01-28 19:00:00| 134|193.60999999999999|            17|
|2020-01-22 19:00:00|  65|            657.03|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|             229.4|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|             165.9|             9|
|2020-01-17 10:00:00|  41| 638.2700000000002|            49|
|2020-01-02 11:00:00|  26|198.60000000000002|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|            148.89|            13|
|2020-01-04 20:00:00|  2

In [29]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [23]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [24]:
df_yellow.createOrReplaceTempView('yellow')

In [25]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    yellow
WHERE
    pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [26]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-09 01:00:00| 188|122.71000000000001|             3|
|2038-02-17 20:00:00| 263|               7.8|             1|
|2020-03-08 15:00:00| 158|2283.1299999999997|           126|
|2020-05-12 23:00:00| 170|             50.86|             3|
|2020-04-11 16:00:00|  43|            148.82|            10|
|2020-01-17 16:00:00| 132| 29275.85999999998|           468|
|2020-04-22 12:00:00| 161|175.55999999999997|            10|
|2020-04-21 15:00:00| 233|181.45000000000002|            12|
|2020-01-05 16:00:00| 162|           4822.54|           277|
|2029-11-19 10:00:00| 264|               3.8|             1|
|2020-01-10 06:00:00| 132| 9030.950000000008|           195|
|2020-01-04 17:00:00| 161| 7491.190000000013|           471|
|2020-01-02 13:00:00| 107| 2215.039999999998|           143|
|2020-02-20 06:00:00|  4

In [31]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [38]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [39]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [40]:
df_join.show()

+-------------------+----+------------------+--------------------+-----------------+---------------------+
|               hour|zone|      green_amount|green_number_records|    yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
|2020-01-01 00:00:00|   3|              NULL|                NULL|             25.0|                    1|
|2020-01-01 00:00:00|  40|168.98000000000002|                   8|            89.97|                    5|
|2020-01-01 00:00:00|  45|              NULL|                NULL|732.4800000000002|                   42|
|2020-01-01 00:00:00|  47|              13.3|                   1|              8.3|                    1|
|2020-01-01 00:00:00|  51|              17.8|                   2|             31.0|                    1|
|2020-01-01 00:00:00|  68|              NULL|                NULL|7825.070000000012|                  396|
|2020-01-01 00:00:00|  73|           

In [41]:
df_join.write.parquet('data/report/revenue/total')